In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with just a few locations, constructing an ITA-style matrix of price vs start/end dates

In [30]:
# Setup relevant variables
ZIP = '60615'
AGENCYBR = 'ET'
AGENCYIDS = ['E11509', #oak park,
             #'E11548', #bolingbrook
             #'E115CH', #elmwood
             #'E11516', #skokie
             #'E11553', #Nn Lincolnwood 53
            ]
RADIUS = '50'

PICKUP = datetime.date(2017, 11, 6)
DROPOFF = PICKUP+datetime.timedelta(days=14)
FIXEDLENGTH = True
VARYSTART = 0
VARYEND = 2
TIME = '11:00 AM'

In [31]:
%run backend_withdb.ipynb

In [32]:
agencylist = getagencylist((PICKUP,DROPOFF))
AGENCYIDS = [x['code'] for x in agencylist.values()]

In [33]:
populateagenciesdb((PICKUP,DROPOFF))

In [34]:
runparser(agencyids=AGENCYIDS, threads=1);

002 ND |  11/06/2017 11/20/2017 3 ('avis', 'CHIC05', 'Chicago - 18th Street & Wabash Avenue') 4.05
003 ND |  11/06/2017 11/20/2017 4 ('budget', 'CHIC02', 'Chicago') 4.05
005 ND |  11/06/2017 11/20/2017 6 ('budget', 'CHIC08', 'Chicago') 5.11
008 ND |  11/06/2017 11/20/2017 9 ('avis', 'CHIX01', 'Chicago Union Train Station') 5.71
009 ND |  11/06/2017 11/20/2017 9 ('avis', 'CHIX01', 'Chicago Union Train Station') 5.71
014 ND |  11/06/2017 11/20/2017 15 ('avis', 'CHIC01', 'North Loop-downtown') 6.05
015 ND |  11/06/2017 11/20/2017 16 ('budget', 'CHIC09', 'North Loop Budget') 6.05
022 ND |  11/06/2017 11/20/2017 23 ('avis', 'MDWT01', 'Chicago Midway Intl Airport') 7.54
023 ND |  11/06/2017 11/20/2017 24 ('budget', 'MDWT02', 'Midway Airport') 7.54
028 ND |  11/06/2017 11/20/2017 29 ('avis', 'CHIC12', 'Grossinger City Autoplex') 7.94
029 ND |  11/06/2017 11/20/2017 29 ('avis', 'CHIC12', 'Grossinger City Autoplex') 7.94
030 ND |  11/06/2017 11/20/2017 31 ('avis', 'CHIS08', 'Oak Lawn/chicago') 

In [ ]:
res = session.query(Agency).filter(Agency.ag_id == 'E11509').first()
print(res)
res.prices

In [23]:
session.query(Price.car_econ, Price.pickup, Agency.ag_dist, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_comp).count()

98

In [38]:
res = session.query(Price.car_int, Price.pickup, Agency.ag_dist, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_int).all()

In [39]:
import pprint
pprint.pprint(res)

[(271.93, '11/06/2017', 10.97, 'Dd Oak Park'),
 (279.94, '11/06/2017', 22.8, 'Nn-glenview'),
 (282.44, '11/06/2017', 17.76, 'Nn South Evanston N2'),
 (283.25, '11/06/2017', 44.16, 'Gurnee Il Sears'),
 (285.6, '11/06/2017', 18.96, 'Aa Holiday Inn'),
 (285.95, '11/06/2017', 37.57, 'Libertyville'),
 (295.39, '11/06/2017', 44.03, 'Nn-lake Villa'),
 (295.39, '11/06/2017', 44.78, 'Nn-gurnee'),
 (295.39, '11/06/2017', 47.3, 'Nn-round Lake Heights'),
 (295.45, '11/06/2017', 36.51, 'Nn-mundelein'),
 (298.26, '11/06/2017', 37.14, 'Nn-libertyville'),
 (299.18, '11/06/2017', 7.54, 'Dd Gold Coast Auto Body'),
 (299.18, '11/06/2017', 7.87, 'Dd Old Town'),
 (299.18, '11/06/2017', 8.91, 'Dd Bucktown'),
 (299.18, '11/06/2017', 9.61, 'Nn Lincoln Park'),
 (299.18, '11/06/2017', 12.59, 'Nn Uptown'),
 (299.20000000000005, '11/06/2017', 8.08, 'Euro Collission'),
 (302.56, '11/06/2017', 18.57, 'Aa Willowbrook'),
 (302.56, '11/06/2017', 26.2, 'Aa Lockport'),
 (302.56, '11/06/2017', 32.22, 'Aa South Naperville

In [13]:
import pprint
pprint.pprint(res)

[(238.85, '11/03/2017', 24.53, 'Aa Glen Ellyn'),
 (239.69, '11/03/2017', 26.2, 'Aa Lockport'),
 (240.26, '11/03/2017', 24.81, 'Aa Bolingbrook'),
 (241.97000000000003, '11/03/2017', 24.83, 'Aa Lisle'),
 (241.97000000000003, '11/03/2017', 27.09, 'Aa East Naperville'),
 (249.76, '11/03/2017', 21.31, 'Aa South Downers Grove'),
 (252.17, '11/03/2017', 16.04, 'Nn Lincolnwood 53'),
 (252.17, '11/03/2017', 17.54, 'Nn Skokie'),
 (252.17, '11/03/2017', 18.86, 'Morton Grove'),
 (256.68, '11/03/2017', 18.96, 'Nn North Evanston'),
 (259.89, '11/03/2017', 10.97, 'Dd Oak Park'),
 (259.89, '11/03/2017', 14.15, 'Dd Elmwood Park'),
 (259.89, '11/03/2017', 16.04, 'Dd Stone Park'),
 (262.23, '11/03/2017', 17.76, 'Nn South Evanston N2'),
 (263.9, '11/03/2017', 13.64, 'Aa Lagrange'),
 (264.81, '11/03/2017', 18.96, 'Aa Holiday Inn'),
 (275.18, '11/03/2017', 15.24, 'Nn Norwood Park C1'),
 (283.62, '11/03/2017', 18.57, 'Aa Willowbrook'),
 (283.64, '11/03/2017', 27.93, 'Nn-hyatt-deerfield'),
 (286.32, '11/03/20

In [ ]:
agencylist = getagencylist((PICKUP,DROPOFF))

In [ ]:
print(agencylist)

In [ ]:
results = runparser(agencyids = AGENCYIDS, threads=5)

In [ ]:
print(results)

In [ ]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [ ]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))